# COM-480 - Data Visualization | OpenFoodFacts database analysis

In [1]:
import json
import re
import pandas as pd

In [2]:
def percent_missing(df, label: str) -> float:
    """
    Percentage of missing values for a given label
    """
    return len(df[(df[label].values == '') | (df[label].isnull())]) / len(df)

def nth_most_frequent(df: pd.DataFrame, label: str, split=False, n=25):
    """
    nth most frequent values for a given label
    """
    if split: return df[label].str.split(',').explode().value_counts().nlargest(n)
    else: return df[label].value_counts().nlargest(n)

def kth_most_frequent_non_cleaned(df, cleaned_label, origin_label, n=25) -> list[str]:
    return df.loc[((df[cleaned_label] == '') | (df[cleaned_label].isnull())) & (df[origin_label].notnull()), [origin_label]].value_counts().nlargest(n)

In [3]:
FILENAME = '../datasets/products_0.995.csv'
ALLERGENS_TAGS_FILENAME  = 'tags/allergens.json'
COUNTRIES_TAGS_FILENAME  = 'tags/countries.json'
COUNTRIES_CODES_FILENAME = 'tags/countries_codes.json'
PACKAGING_TAGS_FILENAME  = 'tags/packaging.json'

with open(COUNTRIES_TAGS_FILENAME, 'r')  as tags_file: COUNTRIES_TAGS  = json.load(tags_file)
with open(COUNTRIES_CODES_FILENAME, 'r') as tags_file: COUNTRIES_CODES = json.load(tags_file)
with open(ALLERGENS_TAGS_FILENAME, 'r')  as tags_file: ALLERGENS_TAGS  = json.load(tags_file)
with open(PACKAGING_TAGS_FILENAME, 'r')  as tags_file: PACKAGING_TAGS  = json.load(tags_file)

REGEX_PACKAGING = r'\b(?:en:|fr:|es:|it:)?([A-Za-zÀ-ÖØ-öø-ÿ\-\.0-9]+(?: [A-Za-zÀ-ÖØ-öø-ÿ\-\.0-9]+)*)'
REGEX_CATEGORY  = r'\b(?:en:)+([A-Za-zÀ-ÖØ-öø-ÿ\-\.0-9]+(?: [A-Za-zÀ-ÖØ-öø-ÿ\-\.0-9]+)*)'
REGEX_LOCATION  = r'\b(?:en:|fr:|es:|it:)?([A-Za-zÀ-ÖØ-öø-ÿ\-\.0-9]+(?: [A-Za-zÀ-ÖØ-öø-ÿ\-\.0-9]+)*)'
REGEX_ALLERGEN  = r'\b(?:en:|fr:|es:|it:)?([A-Za-zÀ-ÖØ-öø-ÿ\-\.0-9]+(?: [A-Za-zÀ-ÖØ-öø-ÿ\-\.0-9]+)*)'

df = pd.read_csv(FILENAME)

/var/folders/fw/8nv4kg9x2zldk0jghwxdl49r0000gn/T/ipykernel_47051/1252387409.py:17: DtypeWarning: Columns (0,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(FILENAME)


### Cleaning of field `abbreviated_product_name`

In [4]:
df.drop(['abbreviated_product_name'], axis=1, inplace=True)

### Cleaning of fields `packaging`, `packaging_tags` and `packaging_en`
Material and packaging type of the product

The current functions only extract the material (e.g. plastic, metal, ...). 

We could further analyze those fields to analyze packaging type.

In [5]:
def split_packaging(packaging: str) -> list[str]:
    if type(packaging) != str: return []
    return re.findall(REGEX_PACKAGING, packaging)

def packaging_to_material(packaging: str) -> str:
    for material, filters in PACKAGING_TAGS.items():
        if material in packaging: return material
        if any([f in packaging for f in filters]): return material
    return None

def cleaning_packaging_material(packaging_tags: str) -> str:
    packagings = set(map(packaging_to_material, split_packaging(packaging_tags)))
    return ','.join(filter(lambda p: p, packagings))

df['packaging'] = df.apply(lambda r : cleaning_packaging_material(r['packaging_tags']), axis=1)
df.drop(['packaging_tags', 'packaging_en', 'packaging_text'], axis=1, inplace=True)

### Cleaning of fields `brands` and `brands_tags`
Information about the product's brand.

Since `brands_tags` is just a formatted version if `brands` and does not add any information we can drop it.

In [6]:
df.drop(['brands_tags'], axis=1, inplace=True)

### Cleaning of fields `categories`, `categories_tags` and `categories_en`
Food categories of the product (e.g. beverages, meats, ...)

In [7]:
def cleaning_categories_tag(categories_tags: str) -> str:
    if type(categories_tags) != str: return
    categories = re.findall(REGEX_CATEGORY, categories_tags)
    categories = [category.replace('-', ' ') for category in categories]
    return ','.join(categories)

df['categories'] = df.apply(lambda r : cleaning_categories_tag(r['categories_tags']), axis=1)
df.drop(['categories_tags', 'categories_en'], axis=1, inplace=True)

### Cleaning of fields `origins`, `origins_tags` and `origins_en`
Countries of origin of the ingredients

In [8]:
def split_locations(locations: str) -> list[str]:
    if type(locations) != str: return []
    return re.findall(REGEX_LOCATION, locations)

def location_to_country(location: str) -> str:
    if location in COUNTRIES_TAGS.keys(): return location
    if location in COUNTRIES_CODES: return COUNTRIES_CODES[location]
    for country, filters in COUNTRIES_TAGS.items():
        if country in location: return country
        if any([f in location for f in filters]): return country
    return None

def cleaning_origins(origins: str, origins_tags: str, origins_en: str) -> str:
    countries = set(map(location_to_country, split_locations(origins))) \
        | set(map(location_to_country, split_locations(origins_tags))) \
        | set(map(location_to_country, split_locations(origins_en)))
    return ','.join(filter(lambda c: c, countries))

df['countries_ingredients'] = df.apply(lambda r : cleaning_origins(r['origins'], r['origins_tags'], r['origins_en']), axis=1)
df.drop(['origins', 'origins_tags', 'origins_en'], axis=1, inplace=True)

### Cleaning of fields `manufacturing_places` and `manufacturing_places_tags`
Countries where the product is manufactured (e.g. processing, packaging, ...)

In [9]:
def cleaning_manufacturing(manufacturing_places: str, manufacturing_places_tags: str) -> str:
    countries = set(map(location_to_country, split_locations(manufacturing_places))) \
        | set(map(location_to_country, split_locations(manufacturing_places_tags)))
    return ','.join(filter(lambda c: c, countries))

df['countries_manufacturing'] = df.apply(lambda r : cleaning_manufacturing(r['manufacturing_places'], r['manufacturing_places_tags']), axis=1)
df.drop(['manufacturing_places', 'manufacturing_places_tags'], axis=1, inplace=True)

### Cleaning of fields `labels`, `labels_tags` and `labels_en`
Labels of the product.

Quite hard to extract meaningful information, there is a lot of different labels for each product (by country, food category, ...). I just extracted some information here. 

In [10]:
df['is_vegan']      = df['labels_tags'].str.contains('vegan', na=False).astype(int)
df['is_vegetarian'] = df['labels_tags'].str.contains('vegetarian', na=False).astype(int)
df['is_organic']    = df['labels_tags'].str.contains('organic|bio|biologique', na=False).astype(int)
df['is_halal']      = df['labels_tags'].str.contains('halal', na=False).astype(int)
df['is_kosher']     = df['labels_tags'].str.contains('kosher', na=False).astype(int)

df.drop(['labels', 'labels_tags', 'labels_en'], axis=1, inplace=True)

### Cleaning of fields `emb_codes`, `emb_codes_tags` and `first_packaging_code_geo`
EMB Code of the product packaging

The field `emb_codes_tags` is just a formatted version of `emb_codes` so we only keep one. The field `first_packaging_code_geo` is almost always null, so we drop it.

In [11]:
df['emb_codes'] = df['emb_codes_tags']
df.drop(['first_packaging_code_geo', 'emb_codes_tags'], axis=1, inplace=True)

### Cleaning of fields `cities` and `cities_tags`
Field `cities` is always empty, we drop it

Field `cities_tags` seems hard to exploit (lot of various format), we drop it

In [12]:
df.drop(['cities', 'cities_tags'], axis=1, inplace=True)

### Cleaning of field `purchase_places`

In [13]:
def cleaning_purchase_places(purchase_places: str) -> str:
    countries = set(map(location_to_country, split_locations(purchase_places)))
    return ','.join(filter(lambda c: c, countries))

df['purchase_places'] = df.apply(lambda r : cleaning_purchase_places(r['purchase_places']), axis=1)

### Cleaning of fields `countries`, `countries_tags` and `countries_en`
Countries where the product is sold

In [14]:
def cleaning_countries(countries: str, countries_tags: str, countries_en: str) -> str:
    countries = set(map(location_to_country, split_locations(countries))) \
        | set(map(location_to_country, split_locations(countries_tags))) \
        | set(map(location_to_country, split_locations(countries_en)))
    return ','.join(filter(lambda c: c, countries))

df['countries_sold'] = df.apply(lambda r : cleaning_countries(r['countries'], r['countries_tags'], r['countries_en']), axis=1)
df.drop(['countries', 'countries_tags', 'countries_en'], axis=1, inplace=True)

### Cleaning of field `ingredients_analysis_tags`
Creation of new fields using tags
- `0` : No
- `1` : Unknown
- `2` : Yes

In [15]:
def cleaning_palm_oil(ingredients_analysis_tags: str) -> int:
    if type(ingredients_analysis_tags) != str: return 1
    if 'en:palm-oil-free' in ingredients_analysis_tags: return 0
    if 'en:palm-oil-content-unknown' in ingredients_analysis_tags \
        or 'en:may-contain-palm-oil' in ingredients_analysis_tags: return 1
    if 'en:palm-oil' in ingredients_analysis_tags: return 2
    else: return 1

def cleaning_vegetarian(ingredients_analysis_tags: str) -> int:
    if type(ingredients_analysis_tags) != str: return 1
    if 'en:non-vegetarian' in ingredients_analysis_tags: return 0
    if 'en:vegetarian-status-unknown' in ingredients_analysis_tags \
        or 'en:maybe-vegetarian' in ingredients_analysis_tags: return 1
    if 'en:vegetarian' in ingredients_analysis_tags: return 2
    else: return 1

def cleaning_vegan(ingredients_analysis_tags: str) -> int:
    if type(ingredients_analysis_tags) != str: return 1
    if 'en:non-vegan' in ingredients_analysis_tags: return 0
    if 'en:vegan-status-unknown' in ingredients_analysis_tags \
        or 'en:maybe-vegan' in ingredients_analysis_tags: return 1
    if 'en:vegan' in ingredients_analysis_tags: return 2
    else: return 1
    
df['palm_oil'] = df.apply(lambda r : cleaning_palm_oil(r['ingredients_analysis_tags']), axis=1).astype(int)
df['is_vegetarian_alt'] = df.apply(lambda r : cleaning_vegetarian(r['ingredients_analysis_tags']), axis=1).astype(int)
df['is_vegan_alt'] = df.apply(lambda r : cleaning_vegan(r['ingredients_analysis_tags']), axis=1).astype(int)

df.drop(['ingredients_analysis_tags'], axis=1, inplace=True)

### Cleaning of field `allergens`
Allergens contained in the product (e.g. milk, fish, ...)

In [16]:
def split_allergens(allergens: str) -> list[str]:
    if type(allergens) != str: return []
    return re.findall(REGEX_ALLERGEN, allergens)

def tag_to_allergen(tag: str) -> str:
    if tag in ALLERGENS_TAGS: return tag
    for allergen, filters in ALLERGENS_TAGS.items():
        if allergen in tag: return allergen
        if any([f in tag for f in filters]): return allergen
    return None

def cleaning_allergens(allergens: str) -> str:
    allergens = set(map(tag_to_allergen, split_allergens(allergens)))
    return ','.join(filter(lambda a: a, allergens))

df['allergens'] = df.apply(lambda r : cleaning_allergens(r['allergens']), axis=1)
df.drop(['allergens_en'], axis=1, inplace=True)

### Cleaning of fields `traces`, `traces_tags` and `traces_en`

In [17]:
def cleaning_traces_tags(traces_tags: str) -> str:
    if type(traces_tags) != str: return
    TRACES = {
        'bones': ['arete', 'os'],
        'oats': ['avena'],
        'phenylalanine': []
    }
    traces = []
    for trace in traces_tags.split(','):
        if 'en:' in trace: traces.append(trace.replace('en:', ''))
        else:
            for t, filters in TRACES.items():
                if t in trace or any([f in trace for f in filters]):
                    traces.append(t)
                    break
    return ','.join(traces)

df['traces'] = df.apply(lambda r : cleaning_traces_tags(r['traces_tags']), axis=1)
df.drop(['traces_tags', 'traces_en'], axis=1, inplace=True)

### Cleaning of field `no_nutrition_data`
Still don't understand the purpose of this field

In [18]:
df.drop(['no_nutrition_data'], axis=1, inplace=True)

### Cleaning of fields `additives`, `additives_tags`, `additives_n` and `additives_en`
Additives present in the product

In [19]:
df['additives'] = df['additives_en']
df.drop(['additives_n', 'additives_tags', 'additives_en'], axis=1, inplace=True)

### Cleaning of fields `nutriscore_score` and `nutriscore_grade`

In [20]:
def cleaning_nutriscore_grade(nutriscore_grade: str) -> str:
    if type(nutriscore_grade) != str: return None
    else: return nutriscore_grade.upper()

df['nutriscore_fr'] = df.apply(lambda row : cleaning_nutriscore_grade(row['nutriscore_grade']), axis=1)
df.drop(['nutriscore_score', 'nutriscore_grade'], axis=1, inplace=True)

### Cleaning of fields `pnns_groups_1` and `pnns_groups_2`
Those fields are deprecated

In [21]:
df.drop(['pnns_groups_1', 'pnns_groups_2'], axis=1, inplace=True)

### Cleaning of fields `food_groups`, `food_groups_tags` and `food_groups_en`
Food groups of the product

In [22]:
def cleaning_food_groups_tags(food_groups_tags: str) -> str:
    if type(food_groups_tags) != str: return
    return ','.join([group.replace('en:', '') for group in food_groups_tags.split(',')])

df['food_groups'] = df.apply(lambda r : cleaning_food_groups_tags(r['food_groups_tags']), axis=1)
df.drop(['food_groups_tags', 'food_groups_en'], axis=1, inplace=True)

### Cleaning of field `brand_owner`

In [23]:
df.drop(['brand_owner'], axis=1, inplace=True)

### Cleaning of fields `ecoscore_score` and `ecoscore_grade`
We have to keep both fields since the grade is computed from the score **plus** some other factors that are not easy to extract (packaging, labels, ...). 

More information : https://docs.score-environnemental.com/v/en/

In [24]:
df['ecoscore_grade'] = df['ecoscore_grade'].replace(['unknown', 'not-applicable'], None)

### Cleaning of field `nutrient_levels_tags`
Creation of new fields based on tags :
- `0` : Low
- `1` : Moderate
- `2` : High

In [25]:
def cleaning_quantity(nutrient_levels_tags: str, tag: str) -> int:
    if type(nutrient_levels_tags) != str: return -1
    elif f'{tag}-in-low-quantity' in nutrient_levels_tags: return 0
    elif f'{tag}-in-moderate-quantity' in nutrient_levels_tags: return 1
    elif f'{tag}-in-high-quantity' in nutrient_levels_tags: return 2 
    else: return 3

df['sugars_quantity'] = df.apply(lambda r : cleaning_quantity(r['nutrient_levels_tags'], 'sugars'), axis=1)
df['fat_quantity'] = df.apply(lambda r : cleaning_quantity(r['nutrient_levels_tags'], 'fat'), axis=1)
df['saturated_fat_quantity'] = df.apply(lambda r : cleaning_quantity(r['nutrient_levels_tags'], 'saturated_fat'), axis=1)
df['salt_quantity'] = df.apply(lambda r : cleaning_quantity(r['nutrient_levels_tags'], 'salt'), axis=1)
df.drop(['nutrient_levels_tags'], axis=1, inplace=True)

### Cleaning of fields `owner`, `data_quality_errors_tags` and `popularity_tags`
The popularity tags may be useful, but they seems hard to exploit

In [26]:
df.drop(['owner', 'data_quality_errors_tags', 'popularity_tags'], axis=1, inplace=True)

### Cleaning of fields `main_category` and `main_categories_en`
Main food category of the product. 

Fields `main_category` and `main_category_en` have a fuzzy meaning, dropping them and using the first category computed before as the main category.

In [27]:
df['main_category'] = df['categories'].str.split(',').str[0]
df.drop(['main_category_en'], axis=1, inplace=True)

### Cleaning of fields `image_url`, `image_ingredients_url` and `image_nutrition_url`
Remove the prefix `https://images.openfoodfacts.org/images/products/` to reduce size of the dataset, we can just add it later if we need.

In [28]:
df['image_url'] = df['image_url'].str.replace('https://images.openfoodfacts.org/images/products/','', regex=False)
df['image_ingredients_url'] = df['image_ingredients_url'].str.replace('https://images.openfoodfacts.org/images/products/','', regex=False)
df['image_nutrition_url'] = df['image_nutrition_url'].str.replace('https://images.openfoodfacts.org/images/products/','', regex=False)

### Cleaning of fields `energy_100g` and `nutrition-score-fr_100g`
Those fields are deprecated

In [29]:
df.drop(['energy_100g', 'nutrition-score-fr_100g'], axis=1, inplace=True)

### Adding field `completeness_custom`
New measure of completness of row after cleaning. It's normal to have a low index because of the fields about nutriments (protein, carbs, ...)

In [30]:
def filter_by_completeness(completeness: float): 
    return df[df['completeness_custom'] > completeness]

def completeness(row) -> float:
    completeness = 0.0
    for r in row:
        if pd.isnull(r) or (type(r) == str and r == ''): completeness += 1.0
    return 1 - completeness / len(row)

df['completeness_custom'] = df.apply(lambda x: completeness(x), axis=1)

### Saving the cleaned dataset

In [31]:
df.to_csv(FILENAME.replace('.csv', '_cleaned.csv'), index=False)
df.head(10).to_csv(FILENAME.replace('.csv', '_cleaned_preview.csv'), index=False)